In [1]:
%matplotlib inline
#Standard imports:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#Non-standard imports:
import folium
import json
from shapely.geometry import shape, Point
import vincent

### Folium
This is a small notebook that shows some examples how you might use folium to make informative maps.
The library folium is based on a javascript library called leaflet.js, which allows making layered maps. It is based on OpenStreetMaps and does not actually render the whole map but rather creates an html file with some javascript magic, that pulls the map data from OpenStreetMaps dynamically. As a result maps saved as html files with folium are not very large files.

This tutorial will give you some insights into how you can make some maps using folium and data downloaded from the [Motor Vehicle Collisions dataset from NYC opendata](https://data.cityofnewyork.us/Public-Safety/NYPD-Motor-Vehicle-Collisions/h9gi-nx95), which we have subsetted to 1 week worth of data using the tool available with the 'View Data' button.

Additionally we will be using some .geojson files. 
+ [NYC Boroughs](http://data.beta.nyc/dataset/nyc-borough-boundaries)
+ [NYC Community Districts](http://data.beta.nyc/dataset/nyc-community-districts)

These serve to define shapes on our maps which we can give different shades using the choropleth function from folium. We will also be using the library shapely to make some calculations as we will see later.


First let us use Pandas to read in the data we downloaded:

In [2]:
crashes = pd.read_csv('NYPD_Motor_Vehicle_Collisions.tsv', sep='\t')

#### Making a simple Map with Markers
folium offers a simple Map function, which initializes us over a ``location`` [Latitude, Longitude] with a ``zoom_start`` of our choosing. Additionally different tilesets, that give our maps a different basic look, are available. Two popular examples are 'Stamen Toner' and 'Stamen Terrain'. With the ``folium.Marker`` and ``folium.CircleMarker`` you can add scaling markers with popups to your map.

In [35]:
nymap = folium.Map(location=[40.7831, -73.9712], zoom_start =12)
latlon_notnull = crashes[['LATITUDE', 'LONGITUDE']].notnull().any(axis=1)
crashes.ix[latlon_notnull,:].head(500).apply(lambda x: folium.CircleMarker([x['LATITUDE'], x['LONGITUDE']],radius=5 , popup=str(x.name)).add_to(nymap), axis=1)

nymap

Individual markers are great but can get confusing when you have too many of them close to each other. This is where ``folium.MarkerCluster`` comes in. You can add a MarkerCluster to your map and then simply add your markers to the MarkerCluster instead of directly to the map. The MarkerCluster groups nearby markers together and upon click on a cluster draws you closer and reveals the Markers inside the cluster (these can be MarkerClusters).

In [34]:
ny_clustermap = folium.Map(location=[40.7831, -73.9712], zoom_start =12)
ny_cluster  = folium.MarkerCluster()
ny_cluster.add_to(ny_clustermap)
crashes.ix[latlon_notnull,:].head(500).apply(lambda x: 
                                             folium.CircleMarker([x['LATITUDE'], 
                                                            x['LONGITUDE']], radius=5, 
                                                           popup=str(x.name)).add_to(ny_cluster),
                                             axis=1)

ny_clustermap

#### Using a GeoJSON file to make a map with the boroughs of New York City
MarkerClusters are a really convenient and flexible way of aggregating multiple nearby Markers into clusters. Often you are interested in specifying areas on your map for which you want to visualize how many events happened. Here we will make a map of the boroughs of New York City and color each borough by the amount of car crash events in the borough.
For that we use what is called a geoJSON file. 

What the heck is a JSON file? 

A JSON file is a JavaScript Object Notation file. You can think of it as a standardized way of writing nested datastructures to a text file. GeoJSON files usually have a first layer that is called a feature collection. Each feature in the collection represents a polygon on the map. The properties field of each feature contains more information on the polygon, like in our case which borough it represents.

We will use the ``.choropleth`` function of our ``folium.Map`` object to color each polygon by the amounts of accidents that have happened in the borough. ``.choropleth`` takes the path to a geoJSON file, ``key_on`` for the field of each feature that should is equivalent to the identifier we use in our data frame, a counts and feature column of our data and a ``fill_color``.

In [12]:
crashes_by_borough = pd.DataFrame(crashes['BOROUGH'].value_counts())
crashes_by_borough.reset_index(level=0, inplace=True)
crashes_by_borough['index'] = crashes_by_borough['index'].str.title()
choromap = folium.Map(location=[40.7831, -73.9712], zoom_start =10)
choromap.choropleth(geo_path='nycboroughboundaries.geojson', 
                    data=crashes_by_borough, 
                    columns=['index', 'BOROUGH'],
                    key_on='feature.properties.borough', fill_color='BuPu',
                    legend_name='Amount of Vehicle Accidents',
                   reset=True)
choromap

#### Adding Markers to the centers of selected polygons
The library ``shapely`` let's us do calculations on shapes from the geojson file. We will use it to put a marker at the center of each borough on the map with its name.
If you zoomed into our choropleth map from the example before, you might have noticed that boroughs are represented by more than one polygon. Each of these individual polygons is a single feature in the geojson file, the ones belonging to the same borough simply have the same value for the ``feature.properties.borough`` field which folium recognizes and colors them all the same as we specified ``key_on``. Now if we get a marker for the center for each of those, the map will not look nice.

Let us load the geojson file using the load function from the ``json`` module (part of python's standard library).
Instead we will compute the area for each feature from the geoJSON file, as well as its centroid's latitude and longitude and get its borough field. We can read in the geometry field of each feature in the JSON file into shapely's ``shape`` function, returning a polygon with ``.centroid.y`` (latitude) and ``.centroid.x``(longitude)
We make a list for each these three values and make a ``pd.DataFrame`` from them. We group by borough, sort by area, take the largest area polygon for each borough and add a marker using the centroid's latitude and longitude.


In [13]:
with open('nycboroughboundaries.geojson') as f:
    bb_json = json.load(f)
    
boroughnames, lat, lon, shapearea = [], [], [], []

for feature in bb_json['features']:
    centroid = shape(feature['geometry']).centroid
    borough_name = feature['properties']['borough']
    area = shape(feature['geometry']).area
    boroughnames.append(borough_name)
    lat.append(centroid.y)
    lon.append(centroid.x)
    shapearea.append(area)
    
geojsondf = pd.DataFrame({'Borough':boroughnames, 'Latitude':lat, 'Longitude':lon, 'Area':shapearea})
for entry in geojsondf.groupby('Borough'):
    dfhandle = entry[1].sort_values('Area', axis=0)
    latitude    = float(dfhandle.tail(1)['Latitude'])
    longitude   = float(dfhandle.tail(1)['Longitude'])
    boroughname =   str(dfhandle.tail(1)['Borough'].values[0])
    folium.Marker([latitude, longitude], popup=boroughname).add_to(choromap)
choromap

#### Add explanation for GEOJSON example where the names of the polygons are not in your original dataset, so you have to infer them using shapely
In the earlier example we were lucky that our original dataset already had all the borough information in it. In this next example we are using a more finely grained geoJSON map which depicts the community districts. That information is not in our original dataframe, so we have to infer it from the latitude and longitude values from our data. 

We will use json.load again to load in the geoJSON file, read the geometry of each feature into shapely.shape. Each of the polygon objects we get from that has a method ``.contains()`` that takes in a shapely ``Point`` object and returns a boolean. 

We define a function that takes in a json object, a latitude and a longitude and iterates over all features in the JSON file and returns the communityDistrict field if the given latitude and longitude are in one of the features from the json file. 


In [14]:
with open('nyccommunitydistricts.geojson') as f:
    cd_json = json.load(f)

def point_in_geojson_features(geojson, lat,lon):
    point = Point(lon,lat)
    for feature in geojson['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            return feature['properties']['communityDistrict']
    


Then we apply this function to all rows using ``DataFrame.apply`` on all rows that have latitude and longitude data. 
This gives us a ``pd.Series`` object that we can assign to a new row in our new DataFrame (all rows with latitude and longitude data). Now we are only interested in rows that feature accidents that have taken place in a community district, so we will drop the rest. Finally we have to adjust the type of the community district column to int.

In [15]:
community_district_series = crashes.ix[latlon_notnull,:].iloc[:,:].apply(lambda x: point_in_geojson_features(cd_json, x.LATITUDE, x.LONGITUDE), axis=1)
crashes_withcoords = crashes.ix[latlon_notnull,:]
crashes_withcoords['Community_district'] = community_district_series
crashes_withcoords = crashes_withcoords.ix[crashes_withcoords['Community_district'].notnull(),:]
crashes_withcoords.ix[:,'Community_district']  = crashes_withcoords.ix[:,'Community_district'].astype(int)

C:\Users\Tobsecret\Anaconda3\envs\Mapstuff\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Lastly we take the counts of the dataframe's community district column and use a choropleth map to visualize it all.

In [16]:
crashes_by_cd = pd.DataFrame(crashes_withcoords['Community_district'].value_counts())
crashes_by_cd.reset_index(level=0, inplace=True)
choromap_cd = folium.Map(location=[40.7831, -73.9712], zoom_start=10)
choromap_cd.choropleth(geo_path='nyccommunitydistricts.geojson',
                      data=crashes_by_cd,
                      columns=['index','Community_district'],
                      key_on='feature.properties.communityDistrict',
                      fill_color='BuPu',
                      legend_name='Amount of Vehicle Accidents')
choromap_cd

#### Saving a map as an HTML
Finally let's save our map as an .html file, that one could send via e-mail or incorporate on a website. 
Usage of the maps requires an internet connection.
We simply use the ``.save('filename.html')`` method of our map object. 

In [11]:
#choromap_cd.save('accidents_in_nyc_by_community_district.html')

#### Other Cool Stuff you can do with Folium
The popup keyword argument of the ``folium.Marker`` function takes in formatted strings but it can also take in formatted html. This opens the door for custom popups, like graphs or even entire websites/ webapps. 
Unfortunately I have not gotten around to making a graph example with the above dataset yet. I plan on updating this section with an example for the community districts, adding a marker with an individual graph to each community district.
This [talk](https://www.youtube.com/watch?v=kmy-sfm3cC8) by Rob Story (the author of folium and vincent), which inspired parts of this tutorial, has a section around the 25 minute mark where he displays such an example, using the plotting library vincent.
